# Projet 7 - Transfer Learning avec ResNet50

### Introduction

Nous avons précédemment tester le tranfer learning avec le réseau vgg16. Nous voulons observer les résultats pour un tunning partiel et voir si on peut obtenir de meilleurs résultats qu'avec le précédent réseau.

In [1]:
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential, Model
import os
import numpy as np
from sklearn.cross_validation import train_test_split
from keras import optimizers
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score

directory = r'C:\Users\zakis\Documents\OpenClassroom\Projet 7\Images'
n_class = 10#♣len(os.listdir(directory))

nb_photos = 1000
Y = []
X = []
for i in range(0, n_class):#len(os.listdir(directory))):
    print(str(i))
    folder = directory + '\\' + os.listdir(directory)[i]
    nb = min(nb_photos, len(os.listdir(folder)))
    for j in range(0, nb):
        file = os.listdir(folder)[j]
        img = folder + '\\' + file
        img = load_img(img, target_size=(197, 197))  # Charger l'image
        img = img_to_array(img)  # Convertir en tableau numpy
        img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
        img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16
        y = np.zeros((n_class,1))
        y[i] = 1
        if len(X) == 0:
            X = img
        else:
            X = np.concatenate([X, img])
        Y.append(y)

X = np.array(X)
Y = np.array(Y).reshape((len(Y),n_class))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

C:\Users\zakis\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\zakis\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0
1
2
3
4
5
6
7
8
9


In [2]:
from keras.applications.resnet50 import ResNet50
# Charger VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected
model = ResNet50(weights="imagenet", include_top=False, input_shape=(197,197,3))

new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(n_class, activation='softmax'))

# Compiler le modèle 
new_model.compile(optimizer=optimizers.Adam(lr=1e-4), 
              loss='categorical_crossentropy', 
              metrics=['acc'])
callbacks = [
    EarlyStopping(patience=3),
    ModelCheckpoint('vgg16_simple.h5', save_best_only=True),
]

model_info = new_model.fit(X_train, y_train, 
                           batch_size=10, 
                           epochs=50, 
                           validation_data=(X_test, y_test),
                           callbacks = callbacks)


Train on 1535 samples, validate on 384 samples
Epoch 1/50
1535/1535 [==============================] - 3357s 2s/step - loss: 1.5113 - acc: 0.6508 - val_loss: 1.5821 - val_acc: 0.7240
Epoch 2/50
1535/1535 [==============================] - 3380s 2s/step - loss: 0.1952 - acc: 0.9485 - val_loss: 1.0300 - val_acc: 0.7552
Epoch 3/50
1535/1535 [==============================] - 4545s 3s/step - loss: 0.0682 - acc: 0.9792 - val_loss: 1.5565 - val_acc: 0.6927
Epoch 4/50
1535/1535 [==============================] - 6221s 4s/step - loss: 0.0622 - acc: 0.9824 - val_loss: 1.0013 - val_acc: 0.7891
Epoch 5/50
1535/1535 [==============================] - 3834s 2s/step - loss: 0.0573 - acc: 0.9831 - val_loss: 1.1488 - val_acc: 0.7578
Epoch 6/50
1535/1535 [==============================] - 3671s 2s/step - loss: 0.0326 - acc: 0.9876 - val_loss: 1.1957 - val_acc: 0.7865
Epoch 7/50
1535/1535 [==============================] - 3666s 2s/step - loss: 0.0572 - acc: 0.9850 - val_loss: 1.8846 - val_acc: 0.6823


On voit ici que les résultats sont particulièrement bon 98% de bonnes prédictions. Ceci est bien mieux que les résultats avec VGG16. Voici comment le modèle est construit:

In [2]:
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential, Model
import os
import numpy as np
from sklearn.cross_validation import train_test_split
from keras import optimizers
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from keras.applications.resnet50 import ResNet50
# Charger VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected
model = ResNet50(weights="imagenet", include_top=False, input_shape=(197,197,3))

new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(10, activation='softmax'))

# Compiler le modèle 
new_model.compile(optimizer=optimizers.Adam(lr=1e-4), 
              loss='categorical_crossentropy', 
              metrics=['acc'])
callbacks = [
    EarlyStopping(patience=3),
    ModelCheckpoint('vgg16_simple.h5', save_best_only=True),
]

In [4]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1003530   
Total params: 24,591,242
Trainable params: 24,538,122
Non-trainable params: 53,120
_________________________________________________________________


In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 203, 203, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

On voit que ce réseau a beaucoup plus de couches que le réseau vgg16. C'est parce que ce réseau est un réseau de résiduel et non convolutionnel comme le vgg16. L'avantage que ce réseau va avoir est que le réseau de neurones est plus profond mais qu'il entraine ces données sur approximativement le même nombre de paramètres. Ce qui nous donne les résultats que l'on peut observer ici et qui sont bien meilleurs que les réseaux convolutionnels.